In [2]:
""" Задачи для общего понимания покупок клиентов"""

' Задачи для общего понимания покупок клиентов'

In [3]:
import pandas as pd
import os

In [4]:
def read_file(file_name):
    if not os.path.exists(file_name):
        raise FileNotFoundError(f'Файл {file_name} не найден')
    else:
        return pd.read_csv(file_name)

Есть 2 файла с данными по поведению клиентов:

df_sales - информация о покупках, одна строка - один заказ. Атрибуты:
- sale_id - идентификатор покупки;
- date - дата покупки;
- count_pizza - количество пицц в заказе;
- count_drink - количество напитков в заказе;
- price - стоимость заказа;
- user_id - идентификатор пользователя;

df_web_logs - талица с логами посещения сайта. Атрибуты:
- user_id - идентификатор пользователя;
- page - какая страница сайта была открыта. Типы страниц:
  - m - страница с меню;
  - b - страница с корзиной и подтверждением заказа;
  - p - страница с оплатой заказа;
- date - дата открытия страницы;
- load_time - время загрузки страницы в миллисекундах.

#### 1. Найти 3 самые продаваемые позиции

In [5]:
df = read_file("sales_detail.csv")
df.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [6]:
top_3_products = df.groupby('good')['price'].agg('sum').sort_values(ascending=False)[:3].index.to_list()

print('Топ 3 самых продаваемых товара - ', top_3_products)

Топ 3 самых продаваемых товара -  ['chefs pizza', 'double pepperoni pizza', 'chicken bbq pizza']


#### 2. Время от захода на сайт до покупки. 
Для каждой покупки нужно посчитать время, которое проходит от покупки до первого захода на сайт того же пользователя в течение двух часов перед покупкой. И от полученных значений времени посчитать среднее.

In [7]:
df_sales = read_file('sales.csv')
df_logs = read_file('web_logs.csv')

In [8]:
#Объединяем таблицы с покупками и логами по пользователям,
#оставляем только user_id и даты
merged_df = pd.merge(df_sales[['user_id', 'date']].rename(columns=            {'date':'date_sale'}), df_logs[['user_id', 'date']].rename(columns={'date':'date_log'}), on='user_id', how='inner')

In [9]:
#Преобразуем столбцы в datetime 
merged_df['date_log'] = pd.to_datetime(merged_df['date_log'])
merged_df['date_sale'] = pd.to_datetime(merged_df['date_sale'])

#Удаляем строки, в которых заход на сайт ранее 2 часов до покупки или после покупки
merged_df_filtered = merged_df[(merged_df['date_log'] >= merged_df['date_sale'] - pd.Timedelta(hours=2)) & (merged_df['date_log'] <= merged_df['date_sale'])].copy()

In [10]:
merged_df_filtered.head()

,user_id,date_sale,date_log
0,1c1543,2022-02-04 10:00:24,2022-02-04 09:50:02
1,1c1543,2022-02-04 10:00:24,2022-02-04 09:51:16
2,1c1543,2022-02-04 10:00:24,2022-02-04 09:52:58
3,1c1543,2022-02-04 10:00:24,2022-02-04 09:57:04
4,1c1543,2022-02-04 10:00:24,2022-02-04 09:57:28


In [11]:
#Находим время первого захода на сайт
first_action = merged_df_filtered.groupby(['user_id', 'date_sale'])['date_log'].agg('min').reset_index()
#Считаем время в секундах между временем первого захода на сайт и заказом
first_action['diff'] = (first_action['date_sale'] - first_action['date_log']).dt.total_seconds()

# считаем среднее время и переводим в минуты
avg_time = round(first_action['diff'].mean()/60)

print("Среднее время от первого захода на сайт до покупки: ", avg_time)

Среднее время от первого захода на сайт до покупки:  17


#### 3. Подсчитать как много пользователей возвращается из месяца в месяц и ответить на вопрос какая доля пользователей, совершивших покупку в феврале, совершила покупку и в марте?

In [12]:
df = read_file("sales_detail.csv")
df.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [13]:
#создаем столбец месяцем
df['month'] = pd.to_datetime(df['date']).dt.month

#выделяем уникальных пользователей в феврале и марте
feb_users = set(df[df['month'] == 2]['user_id'])
march_users = set(df[df['month'] == 3]['user_id'])

#пользователи с покупкой в феврале и марте 
common_users = feb_users & march_users

answer = round(len(common_users)/len(feb_users),2)

print('Доля пользователей, совершивших покупку в феврале, совершила покупку и в марте: ', answer)

Доля пользователей, совершивших покупку в феврале, совершила покупку и в марте:  0.66


#### 4. Как часто пользователь заходит на сайт и ничего не покупает в этот день?

In [14]:
df_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [15]:
# добавляем столбец с датой без времени
df_logs['day'] = pd.to_datetime(df_logs['date']).dt.date
df_sales['day'] = pd.to_datetime(df_sales['date']).dt.date

In [16]:
# делаем таблицы с уникальными парами человек-дата
user_day_web = df_logs[['user_id', 'day', 'page']].groupby(['user_id', 'day'])[['page']].count()
user_day_sale = df_sales[['user_id', 'day', 'price']].groupby(['user_id', 'day'])[['price']].sum()

In [17]:
len(user_day_web), len(user_day_sale)

(303127, 203155)

In [18]:
user_day_web.head()

page
user_id day             
000096  2022-03-04     8
        2022-03-22     7
0000d4  2022-02-28     9
        2022-03-27    10
0000de  2022-02-11     6

In [19]:
user_day_sale.head()

price
user_id day              
000096  2022-03-04    720
        2022-03-22    720
0000d4  2022-02-28    720
        2022-03-27    720
0000de  2022-02-11    660

In [20]:
# соединяем таблицы
df_join = pd.merge(
    user_day_web,
    user_day_sale,
    left_index=True,
    right_index=True,
    how='outer'
)

In [21]:
# считаем доли нулей в столбцах
df_join.isnull().mean()

page     0.000000
price    0.329802
dtype: float64

**Вывод:** 
- Если делал покупку, то заходил на сайт в этот день;
- Если заходил на сайт, то не обязательно покупал;
- Доля человеко-дней без покупок 33%.